## **Setting up Development Environment**

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
% cd /content/drive/MyDrive/Colab Notebooks/Hate-Speech-Detection-in-Tamil/HASOC Tamil/Task 1

/content/drive/MyDrive/Colab Notebooks/Hate-Speech-Detection-in-Tamil/HASOC Tamil/Task 1


In [3]:
! pip install -U -q transformers
! pip install -U -q sentencepiece

     |████████████████████████████████| 3.4 MB 7.5 MB/s 
     |████████████████████████████████| 596 kB 61.2 MB/s 
     |████████████████████████████████| 67 kB 6.1 MB/s 
     |████████████████████████████████| 3.3 MB 15.0 MB/s 
     |████████████████████████████████| 895 kB 67.9 MB/s 
     |████████████████████████████████| 1.2 MB 7.6 MB/s 


## **Importing Libraries**

In [26]:
import pandas as pd

import numpy as np

from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import SGDClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

from transformers import AutoTokenizer, AutoModel

import pickle

## **Loading Data**

In [5]:
dataframe = pd.read_csv("Dataset/tamil_offensive_train.tsv", sep='\t')

In [6]:
dataframe.head()

,id,text,category
0,tam1,திருமலை நாயக்கர் பேரவை சார்பாக படம் வெற்றி பெற...,NOT
1,tam2,இந்த ட்ரெய்லர் கூட பார்க்கிற மாதிரி இல்லை.. இத...,OFF
2,tam3,மைசூரு செட்டியார் சமூகத்தின் சார்பாக இப்படம் வ...,NOT
3,tam4,மொத்த சாதியும் ஒரு சாதிக்கு எதிரா நிக்குது.......,OFF
4,tam5,only for விஜய் சேதுபதி and STR,NOT


In [7]:
dataframe.describe()

,id,text,category
count,5880,5880,5880
unique,5880,5812,3
top,tam1692,வந்தா ராஜாவா தான் வருவேன்........ ராஜாவா வந்தா...,NOT
freq,1,5,4724


In [8]:
dataframe['category'].unique()

array(['NOT', 'OFF', 'not-Tamil'], dtype=object)

In [9]:
dataframe.shape

(5880, 3)

In [10]:
dataframe = dataframe.loc[dataframe['category'] != 'not-Tamil']
dataframe['category'].unique()

array(['NOT', 'OFF'], dtype=object)

In [11]:
dataframe.shape

(5877, 3)

In [12]:
text = dataframe['text']

In [13]:
text

0       திருமலை நாயக்கர் பேரவை சார்பாக படம் வெற்றி பெற...
1       இந்த ட்ரெய்லர் கூட பார்க்கிற மாதிரி இல்லை.. இத...
2       மைசூரு செட்டியார் சமூகத்தின் சார்பாக இப்படம் வ...
3       மொத்த சாதியும் ஒரு சாதிக்கு எதிரா நிக்குது.......
4                          only for விஜய் சேதுபதி and STR
                              ...                        
5875    இப்போதான் ஜாதி பிரச்சினை இல்லாம இருக்கு இப்போ ...
5876    இது ஒரு சாதி ரீதியான படம் இல்லை தான் செல்லும் ...
5877    கதை தெரியவில்லை  இருந்தாலும் மாறுபட்ட சிந்தனை ...
5878    அட பரவால்லையே 8.5 டிஸ்லைக்ஸ் போட்ருக்கானுங்க.....
5879    உதயநிதிக்கு நடிப்பு வரலனு சொல்ரவங்க லைக் போடுங...
Name: text, Length: 5877, dtype: object

In [14]:
label = dataframe['category']

In [15]:
label

0       NOT
1       OFF
2       NOT
3       OFF
4       NOT
       ... 
5875    NOT
5876    NOT
5877    NOT
5878    OFF
5879    NOT
Name: category, Length: 5877, dtype: object

## **Data Preprocessing**

In [16]:
le = LabelEncoder()
label = le.fit_transform(label)
label

array([0, 1, 0, ..., 0, 1, 0])

In [17]:
text = text.str.replace(r"[+/#@&*$%:]",'')
text = text.to_numpy()
text

array(['திருமலை நாயக்கர் பேரவை சார்பாக படம் வெற்றி பெற வாழ்த்துக்கள்',
       'இந்த ட்ரெய்லர் கூட பார்க்கிற மாதிரி இல்லை.. இதை தியேட்டர் ல போய் பார்க்கனுமா.. .',
       'மைசூரு செட்டியார் சமூகத்தின் சார்பாக இப்படம் வெற்றிபெற வாழ்த்துகிறேன்',
       ...,
       'கதை தெரியவில்லை  இருந்தாலும் மாறுபட்ட சிந்தனை வாழ்த்துக்கள்  பறையர் சார்பாக  நன்றி மோகன் ஜி',
       'அட பரவால்லையே 8.5 டிஸ்லைக்ஸ் போட்ருக்கானுங்க.. பரவால வயிறு எறிஞ்சி சாவுங்கடா சாவுங்கடா சாக்காடை நாய்ங்களா..இன்னும் நிறைய டிஸ்லைக்ஸ் எதிர் பாக்குறன்..',
       'உதயநிதிக்கு நடிப்பு வரலனு சொல்ரவங்க லைக் போடுங்கள் ...'],
      dtype=object)

## **Feature Extraction**

In [27]:
# Load Transformer Model

tokenizer = AutoTokenizer.from_pretrained("bert-base-multilingual-cased")
model = AutoModel.from_pretrained("bert-base-multilingual-cased")

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [28]:
# Sample Output

tokenized_input = tokenizer(
        text[0],
        padding=True,
        truncation=False,
        return_tensors='pt'
    )

sample_output = model(**tokenized_input)

In [29]:
print(tokenized_input)

{'input_ids': tensor([[   101,  55993,  53500, 108387,   1160,  15472,  15220,  19932,  14833,
           1162,  18827,  21060,  20654,   1154,  18191,  88626,  19894,   1162,
          37001,   1170, 111312,  50906,   1162, 111312,  30746,   1170,  15472,
          58214, 107350,  11774,    102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1]])}


In [30]:
print(sample_output)

BaseModelOutputWithPoolingAndCrossAttentions(last_hidden_state=tensor([[[ 0.0743,  0.1025, -0.1230,  ...,  0.2188,  0.1360, -0.0136],
         [-0.2465,  0.1869,  0.7358,  ...,  0.7698,  0.5281,  0.6979],
         [ 0.0760, -0.0200,  0.2795,  ...,  0.8895,  0.5187,  0.6248],
         ...,
         [ 0.1687,  0.3453,  0.4591,  ...,  0.3758,  0.2382,  0.2455],
         [-0.6852, -0.1601, -0.5097,  ...,  0.1947,  0.0025,  0.0241],
         [ 0.0169, -0.0398,  0.0454,  ...,  0.2945,  0.2946,  0.0964]]],
       grad_fn=<NativeLayerNormBackward0>), pooler_output=tensor([[ 2.5703e-01,  3.9557e-03,  2.6127e-01, -4.5412e-03,  6.7911e-02,
          2.5216e-01,  2.8130e-02,  1.0552e-01, -2.1710e-01,  1.9736e-01,
          1.0938e-01, -1.4123e-01, -1.3799e-02,  3.5553e-02,  2.0998e-01,
         -1.0093e-01,  3.9862e-01, -3.8515e-02,  4.7670e-03, -2.1881e-01,
         -9.9998e-01, -1.6530e-01, -1.0362e-01, -2.8485e-02, -2.8793e-01,
         -1.3170e-01, -1.4038e-01,  4.1965e-02,  1.5626e-01, -3.529

In [31]:
print(sample_output.pooler_output.cpu().detach().numpy().shape)

(1, 768)


In [32]:
# Tokenizing Input Data

input = []

for i in text:
    tokenized_input = tokenizer(
        i,
        padding=True,
        truncation=False,
        return_tensors='pt'
    )
    input.append(tokenized_input)

In [34]:
# Embedding the Input Data

output = []

j = 1

for i in range(len(input)):
    
    model_output = model(**input[i])
    model_output = model_output.pooler_output.cpu().detach().numpy()
    output.append(model_output)
    
    # Saving Embedded Input Data to Disk

    if ((i + 1) % 200) == 0:
        
        file_name = "./TrainData/output" + str(j) + ".pickle"
        
        with open(file_name, "wb") as fp:   
            # Pickling
            pickle.dump(output, fp)
        
        print(file_name + " done")
        output = []
        j += 1
        
file_name = "./TrainData/output" + str(j) + ".pickle"

with open(file_name, "wb") as fp:   
    # Pickling
    pickle.dump(output, fp)

print(file_name + " done")
output = []

./TrainData/output1.pickle done
./TrainData/output2.pickle done
./TrainData/output3.pickle done
./TrainData/output4.pickle done
./TrainData/output5.pickle done
./TrainData/output6.pickle done
./TrainData/output7.pickle done
./TrainData/output8.pickle done
./TrainData/output9.pickle done
./TrainData/output10.pickle done
./TrainData/output11.pickle done
./TrainData/output12.pickle done
./TrainData/output13.pickle done
./TrainData/output14.pickle done
./TrainData/output15.pickle done
./TrainData/output16.pickle done
./TrainData/output17.pickle done
./TrainData/output18.pickle done
./TrainData/output19.pickle done
./TrainData/output20.pickle done
./TrainData/output21.pickle done
./TrainData/output22.pickle done
./TrainData/output23.pickle done
./TrainData/output24.pickle done
./TrainData/output25.pickle done
./TrainData/output26.pickle done
./TrainData/output27.pickle done
./TrainData/output28.pickle done
./TrainData/output29.pickle done
./TrainData/output30.pickle done


In [35]:
# Loading Embedded Input Data from Disk

output = []

for i in range(30):
    file_name = "./TrainData/output" + str(i + 1) + ".pickle"
    with open(file_name, "rb") as fp:   #Pickling
        file_output = pickle.load(fp)
        for x in file_output:
            output.append(x)
    print(file_name + " done")

X = output
output = []

./TrainData/output1.pickle done
./TrainData/output2.pickle done
./TrainData/output3.pickle done
./TrainData/output4.pickle done
./TrainData/output5.pickle done
./TrainData/output6.pickle done
./TrainData/output7.pickle done
./TrainData/output8.pickle done
./TrainData/output9.pickle done
./TrainData/output10.pickle done
./TrainData/output11.pickle done
./TrainData/output12.pickle done
./TrainData/output13.pickle done
./TrainData/output14.pickle done
./TrainData/output15.pickle done
./TrainData/output16.pickle done
./TrainData/output17.pickle done
./TrainData/output18.pickle done
./TrainData/output19.pickle done
./TrainData/output20.pickle done
./TrainData/output21.pickle done
./TrainData/output22.pickle done
./TrainData/output23.pickle done
./TrainData/output24.pickle done
./TrainData/output25.pickle done
./TrainData/output26.pickle done
./TrainData/output27.pickle done
./TrainData/output28.pickle done
./TrainData/output29.pickle done
./TrainData/output30.pickle done


In [36]:
X = np.array(X)
X.shape

(5877, 1, 768)

In [37]:
X = X.reshape(5877, 768)
X.shape

(5877, 768)

In [38]:
y = label
y.shape

(5877,)

## **Train Test Split**

In [39]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

In [40]:
X_train.shape

(4701, 768)

In [41]:
X_val.shape

(1176, 768)

## **Logistic Regression**

In [43]:
lr = LogisticRegression(max_iter=1000)
lr.fit(X_train, y_train)
lr_pred = lr.predict(X_val)
print(classification_report(y_val, lr_pred))

              precision    recall  f1-score   support

           0       0.85      0.95      0.90       950
           1       0.57      0.28      0.37       226

    accuracy                           0.82      1176
   macro avg       0.71      0.61      0.63      1176
weighted avg       0.79      0.82      0.80      1176



## **Support Vector Machine**

In [44]:
svm = SVC()
svm.fit(X_train, y_train)
svm_pred = svm.predict(X_val)
print(classification_report(y_val, svm_pred))

              precision    recall  f1-score   support

           0       0.81      1.00      0.89       950
           1       0.00      0.00      0.00       226

    accuracy                           0.81      1176
   macro avg       0.40      0.50      0.45      1176
weighted avg       0.65      0.81      0.72      1176



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## **Naive Bayes**

In [45]:
nb = GaussianNB()
nb.fit(X_train, y_train)
nb_pred = nb.predict(X_val)
print(classification_report(y_val, nb_pred))

              precision    recall  f1-score   support

           0       0.83      0.89      0.86       950
           1       0.32      0.22      0.26       226

    accuracy                           0.76      1176
   macro avg       0.57      0.55      0.56      1176
weighted avg       0.73      0.76      0.74      1176



## **Stochastic Gradient Descent**

In [46]:
sgd = SGDClassifier()
sgd.fit(X_train, y_train)
sgd_pred = sgd.predict(X_val)
print(classification_report(y_val, sgd_pred))

              precision    recall  f1-score   support

           0       0.84      0.94      0.89       950
           1       0.53      0.27      0.36       226

    accuracy                           0.81      1176
   macro avg       0.69      0.61      0.62      1176
weighted avg       0.78      0.81      0.79      1176



## **K Nearest Neighbours**

In [47]:
knn = KNeighborsClassifier()
knn.fit(X_train, y_train)
knn_pred = knn.predict(X_val)
print(classification_report(y_val, knn_pred))

              precision    recall  f1-score   support

           0       0.83      0.93      0.88       950
           1       0.43      0.22      0.29       226

    accuracy                           0.79      1176
   macro avg       0.63      0.57      0.58      1176
weighted avg       0.76      0.79      0.77      1176



## **Decision tree**

In [48]:
dt = DecisionTreeClassifier()
dt.fit(X_train, y_train)
dt_pred = dt.predict(X_val)
print(classification_report(y_val, dt_pred))

              precision    recall  f1-score   support

           0       0.83      0.82      0.83       950
           1       0.30      0.31      0.31       226

    accuracy                           0.73      1176
   macro avg       0.57      0.57      0.57      1176
weighted avg       0.73      0.73      0.73      1176



## **Random Forest**

In [49]:
rf = RandomForestClassifier()
rf.fit(X_train, y_train)
rf_pred = rf.predict(X_val)
print(classification_report(y_val, rf_pred))

              precision    recall  f1-score   support

           0       0.82      0.99      0.89       950
           1       0.56      0.08      0.14       226

    accuracy                           0.81      1176
   macro avg       0.69      0.53      0.52      1176
weighted avg       0.77      0.81      0.75      1176



## **Majority Voting**

In [50]:
mv_pred = []

for i in range(len(lr_pred)):
    one = 0
    zero = 0
    # predictions = [rf_pred[i], knn_pred[i], dt_pred[i], lr_pred[i], svm_pred[i], sgd_pred[i], nb_pred[i]]
    predictions = [knn_pred[i], dt_pred[i], lr_pred[i], sgd_pred[i], nb_pred[i]]
    for pred in predictions:
        if pred == 1: one += 1
        if pred == 0: zero +=1
    if one > zero: mv_pred.append(1)
    else: mv_pred.append(0)

mv_pred = np.array(mv_pred)

print(classification_report(y_val, mv_pred))

              precision    recall  f1-score   support

           0       0.84      0.96      0.90       950
           1       0.59      0.22      0.32       226

    accuracy                           0.82      1176
   macro avg       0.71      0.59      0.61      1176
weighted avg       0.79      0.82      0.79      1176



## **Loading Testing Data**

In [51]:
dataframe_test = pd.read_csv(
    "Dataset/tam_offesive_withoutlabels_test.tsv", 
    sep='\t'
)
dataframe_test.head()

,id,text
0,tam_test_1,தலைவா STR இதுக்குதான் கதுருந்தோம் மாஸ் தலைவா t...
1,tam_test_2,நாயுடு மக்கள் சார்பாக திரைப்படம் வெற்றி பெற வா...
2,tam_test_3,தில்லானா முயற்சி தஞ்சை கோனார்
3,tam_test_4,திரௌபதியின் துகிலுறித்த போது காத்த கண்ணனின் யா...
4,tam_test_5,நான் தியேட்டர்லே படம் பார்த்து 35 வருஷமாச்சு! ...


In [52]:
dataframe_test.describe()

,id,text
count,654,654
unique,654,654
top,tam_test_589,"தமிழக வரலாற்றில், முதல் (1) இடம் பிடித்த திரை..."
freq,1,1


In [53]:
text_test = dataframe_test['text']

## **Test Data Preprocessing**

In [54]:
text_test = text_test.str.replace(r"[+/#@&*$%:]",'')
text_test = text_test.to_numpy()

## **Test Feature Extraction**

In [55]:
# Load Transformer Model

tokenizer = AutoTokenizer.from_pretrained("bert-base-multilingual-cased")
model = AutoModel.from_pretrained("bert-base-multilingual-cased")

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [56]:
# Sample Output

tokenized_input = tokenizer(
        text_test[0],
        padding=True,
        truncation=False,
        return_tensors='pt'
    )

sample_output = model(**tokenized_input)

In [57]:
print(tokenized_input)

{'input_ids': tensor([[  101,  1159, 22338, 31829, 15472, 53317, 11273, 17369, 18065, 85591,
          1152, 14124, 53500, 17002, 26225, 12520,  1163, 15472, 18902,  1159,
         22338, 31829, 15472, 11084, 10174, 11703, 13774, 10477,  1157, 46492,
         12520,   102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1]])}


In [58]:
print(sample_output)

BaseModelOutputWithPoolingAndCrossAttentions(last_hidden_state=tensor([[[ 0.0303, -0.2064, -0.1048,  ...,  0.2061,  0.2624,  0.0137],
         [-0.3909, -0.6082,  0.4598,  ...,  0.9541,  0.4150,  0.0942],
         [-0.3604, -0.4333,  0.0177,  ...,  1.1636,  0.4226,  0.1600],
         ...,
         [ 0.1028, -0.9377, -0.2775,  ...,  0.0215,  0.2719,  0.1561],
         [ 0.1615, -1.0197,  0.7045,  ...,  0.2959, -0.0580,  0.4065],
         [ 0.0256, -0.6029,  0.3840,  ...,  0.2838,  0.3579,  0.1613]]],
       grad_fn=<NativeLayerNormBackward0>), pooler_output=tensor([[ 9.3219e-02, -1.8786e-02,  2.2402e-01,  1.2791e-02,  6.2641e-02,
          2.0258e-01,  2.4107e-02,  1.1955e-01, -1.9897e-01,  1.0534e-01,
          4.3506e-02, -5.4878e-02, -3.1530e-02, -2.6551e-02,  6.5359e-02,
         -2.1326e-01,  3.9489e-01, -2.6856e-02,  1.6998e-01, -2.4296e-01,
         -9.9997e-01, -1.0039e-01,  6.7488e-02,  9.1489e-02, -3.6447e-01,
         -8.2296e-02, -4.8892e-02, -4.0027e-03,  6.5776e-03, -4.164

In [59]:
print(sample_output.pooler_output.cpu().detach().numpy().shape)

(1, 768)


In [61]:
# Tokenizing Test Input Data

input = []

for i in text_test:
    tokenized_input = tokenizer(
        i,
        padding=True,
        truncation=True,
        return_tensors='pt'
    )
    input.append(tokenized_input)

In [62]:
# Embedding the Test Input Data

output = []

j = 1

for i in range(len(input)):
    
    model_output = model(**input[i])
    model_output = model_output.pooler_output.cpu().detach().numpy()
    output.append(model_output)
    
    # Saving Embedded Test Input Data to Disk

    if ((i + 1) % 200) == 0:
        
        file_name = "./TestData/output" + str(j) + ".pickle"
        
        with open(file_name, "wb") as fp:   
            # Pickling
            pickle.dump(output, fp)
        
        print(file_name + " done")
        output = []
        j += 1
        
file_name = "./TestData/output" + str(j) + ".pickle"

with open(file_name, "wb") as fp:   
    # Pickling
    pickle.dump(output, fp)

print(file_name + " done")
output = []

./TestData/output1.pickle done
./TestData/output2.pickle done
./TestData/output3.pickle done
./TestData/output4.pickle done


In [63]:
# Loading Embedded Input Test Data from Disk

output = []

for i in range(4):
    file_name = "./TestData/output" + str(i + 1) + ".pickle"
    with open(file_name, "rb") as fp:   #Pickling
        file_output = pickle.load(fp)
        for x in file_output:
            output.append(x)
    print(file_name + " done")

X_test = output
output = []

./TestData/output1.pickle done
./TestData/output2.pickle done
./TestData/output3.pickle done
./TestData/output4.pickle done


In [64]:
X_test = np.array(X_test)
X_test.shape

(654, 1, 768)

In [65]:
X_test = X_test.reshape(654, 768)
X_test.shape

(654, 768)

## **Getting Predictions**

In [66]:
# Logistic Regression
lr_test_pred = lr.predict(X_test)

# Support Vector Machine
svm_test_pred = svm.predict(X_test)

# Naive Bayes
nb_test_pred = nb.predict(X_test)

# Stochastic Gradient Descent
sgd_test_pred = sgd.predict(X_test)

# K Nearest Neighbours
knn_test_pred = knn.predict(X_test)

# Decision Tree
dt_test_pred = dt.predict(X_test)

# Random Forest
rf_test_pred = rf.predict(X_test)

In [67]:
# Majority Voting
mv_test_pred = []

for i in range(len(lr_test_pred)):
    one = 0
    zero = 0
    # predictions = [lr_test_pred[i], svm_test_pred[i], nb_test_pred[i], 
    #                sgd_test_pred[i], knn_test_pred[i], dt_test_pred[i], rf_test_pred[i]]
    predictions = [lr_test_pred[i], svm_test_pred[i], nb_test_pred[i], 
                   sgd_test_pred[i], dt_test_pred[i]]
    for pred in predictions:
        if pred == 1: one += 1
        if pred == 0: zero +=1
    if one > zero: mv_test_pred.append(1)
    else: mv_test_pred.append(0)

mv_test_pred = np.array(mv_test_pred)

## **Evaluating the Predictions**

In [68]:
dataframe_eval = pd.read_csv(
    "Dataset/tamil_offensive_test task 1 with labels.tsv", 
    sep='\t'
)

In [69]:
y_test = dataframe_eval['category']
y_test = LabelEncoder().fit_transform(y_test)

In [70]:
print('Logistic Regression')
print(classification_report(y_test, lr_test_pred))

Logistic Regression
              precision    recall  f1-score   support

           0       0.85      0.97      0.91       536
           1       0.63      0.22      0.33       118

    accuracy                           0.84       654
   macro avg       0.74      0.60      0.62       654
weighted avg       0.81      0.84      0.80       654



In [71]:
print('Support Vector Machine')
print(classification_report(y_test, svm_test_pred))

Support Vector Machine
              precision    recall  f1-score   support

           0       0.82      1.00      0.90       536
           1       0.00      0.00      0.00       118

    accuracy                           0.82       654
   macro avg       0.41      0.50      0.45       654
weighted avg       0.67      0.82      0.74       654



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [72]:
print('Naive Bayes')
print(classification_report(y_test, nb_test_pred))

Naive Bayes
              precision    recall  f1-score   support

           0       0.84      0.90      0.87       536
           1       0.31      0.20      0.24       118

    accuracy                           0.77       654
   macro avg       0.57      0.55      0.56       654
weighted avg       0.74      0.77      0.75       654



In [73]:
print('Stochastic Gradient Descent')
print(classification_report(y_test, sgd_test_pred))

Stochastic Gradient Descent
              precision    recall  f1-score   support

           0       0.85      0.96      0.90       536
           1       0.52      0.21      0.30       118

    accuracy                           0.82       654
   macro avg       0.68      0.58      0.60       654
weighted avg       0.79      0.82      0.79       654



In [74]:
print('K Nearest Neighbours')
print(classification_report(y_test, knn_test_pred))

K Nearest Neighbours
              precision    recall  f1-score   support

           0       0.84      0.94      0.89       536
           1       0.40      0.19      0.25       118

    accuracy                           0.80       654
   macro avg       0.62      0.56      0.57       654
weighted avg       0.76      0.80      0.77       654



In [75]:
print('Decision Tree')
print(classification_report(y_test, dt_test_pred))

Decision Tree
              precision    recall  f1-score   support

           0       0.85      0.82      0.83       536
           1       0.29      0.34      0.31       118

    accuracy                           0.73       654
   macro avg       0.57      0.58      0.57       654
weighted avg       0.75      0.73      0.74       654



In [76]:
print('Random Forest')
print(classification_report(y_test, rf_test_pred))

Random Forest
              precision    recall  f1-score   support

           0       0.83      0.98      0.90       536
           1       0.44      0.07      0.12       118

    accuracy                           0.82       654
   macro avg       0.64      0.52      0.51       654
weighted avg       0.76      0.82      0.76       654



In [77]:
print('Majority Voting')
print(classification_report(y_test, mv_test_pred))

Majority Voting
              precision    recall  f1-score   support

           0       0.83      0.99      0.90       536
           1       0.60      0.10      0.17       118

    accuracy                           0.83       654
   macro avg       0.72      0.54      0.54       654
weighted avg       0.79      0.83      0.77       654

